In [3]:
import pandas as pd
import numpy as np 

In [4]:
df = pd.read_csv('1004_df.csv')

df['date'] = df['date'].astype(int)

df['gender'].replace({'M':0, 'F':1}, inplace=True)

age_list = []
for age in df['age']:
    if age[1:] == '0e':
        age = age[0] + '0'
    else:
        age = age[0] + '5'

    # age_list.append(int(age))
    age_list.append(age)

df['age'] = age_list

null_values = df[df['lf04r'].isnull()]
not_null_values = df[df['lf04r'].notnull()]

df['UseLife'] = 9999
df.loc[null_values.index, 'UseLife'] = 0
df.loc[not_null_values.index, 'UseLife'] = 1

df.drop(columns=['구명'], inplace=True)

df['UseLife'] = 9999
df.loc[null_values.index, 'UseLife'] = 0
df.loc[not_null_values.index, 'UseLife'] = 1

df.drop(columns=['광역시도명', '시군구명', '행정동명'], inplace=True)

/tmp/ipykernel_203766/3771056205.py:1: DtypeWarning: Columns (435) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('1004_df.csv')


In [ ]:
# 결측치가 존재하지 않은 완전한 데이터셋
df_default = df.dropna(axis=1)

# 스케일링
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm

scaler = MinMaxScaler()
df_default_scaled = pd.DataFrame(scaler.fit_transform(df_default), columns=df_default.columns)

## 유의 변수 찾기

In [ ]:
# 후진 제거법 -> 신한라이프 고객 여부를 기준으로 유의한 변수 찾기

In [6]:
def backward_elimination_with_p_value(X, y, significance_level=0.05):
    while True:
        model = sm.Logit(y, X).fit(disp=False)
        max_p_value = max(model.pvalues)
        if max_p_value > significance_level:
            remove_feature_index = model.pvalues.idxmax()
            X = X.drop(remove_feature_index, axis=1)
            print(f"Removed feature: {remove_feature_index}, p-value: {max_p_value}")
        else:
            break
    return X

# 종속 변수 및 독립 변수 설정
X = df_default_scaled.iloc[:,:-1]
y = df_default_scaled['UseLife']

# 후진 제거법 적용
X_selected = backward_elimination_with_p_value(X, y)

# # 최종 선택된 독립 변수 출력
# print("Final Selected Features:")
# print(X_selected)

Removed feature: bd19r, p-value: 0.9706159261513303
Removed feature: be07r, p-value: 0.9574896970035353
Removed feature: bd23r, p-value: 0.8666931460200973
Removed feature: bb05r, p-value: 0.7971256277786031
Removed feature: bd22m, p-value: 0.7059932430087182
Removed feature: bc05m, p-value: 0.663747665552618
Removed feature: ba13r, p-value: 0.599285294998223
Removed feature: bd14m, p-value: 0.4905185502546503
Removed feature: bc08m, p-value: 0.4812714499523334
Removed feature: bd18m, p-value: 0.46071298661058846
Removed feature: ba03r, p-value: 0.3811031197129334
Removed feature: bb02m, p-value: 0.3742818768911603
Removed feature: ba14m, p-value: 0.315954621551637
Removed feature: bd21r, p-value: 0.2096387776180917
Removed feature: bb10m, p-value: 0.19557931458014066
Removed feature: ba10m, p-value: 0.17864567360228822
Removed feature: ba06m, p-value: 0.16509371820170438
Removed feature: bc09m, p-value: 0.1573642285337855
Removed feature: bd20m, p-value: 0.20263459469852374
Removed fe

In [7]:
model = sm.Logit(y, X_selected).fit()

# 모델 요약 출력
print(model.summary())

Optimization terminated successfully.
         Current function value: 0.276780
         Iterations 8
                           Logit Regression Results                           
Dep. Variable:                UseLife   No. Observations:               114600
Model:                          Logit   Df Residuals:                   114550
Method:                           MLE   Df Model:                           49
Date:                Sat, 07 Oct 2023   Pseudo R-squ.:                  0.4387
Time:                        12:01:21   Log-Likelihood:                -31719.
converged:                       True   LL-Null:                       -56509.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
date           1.1976      0.024     49.076      0.000       1.150       1.245
gender         0.4928      0.

In [9]:
X_selected

,pk1,pk2,pk3,pk4,ba01m,ba02m,ba04m,ba05r,ba08m,ba09r,...,bd25r,bd27r,bd28m,bd29r,be01r,be02r,be03r,be04r,be05r,be06r
0,0.0,1.0,0.0,0.000000e+00,0.050708,0.084171,0.000685,0.740260,0.037797,0.148148,...,0.475610,0.666667,0.002436,0.455882,0.819672,0.233333,0.063830,0.152542,0.716216,0.229508
1,0.0,1.0,0.0,4.918075e-07,0.121913,0.178550,0.000000,0.701299,0.101643,0.222222,...,0.426829,0.705128,0.004360,0.500000,0.836066,0.500000,0.106383,0.135593,0.513514,0.475410
2,0.0,1.0,0.0,8.196792e-07,0.039515,0.045622,0.000000,0.727273,0.027252,0.296296,...,0.536585,0.756410,0.001678,0.735294,0.786885,0.416667,0.106383,0.186441,0.540541,0.442623
3,0.0,1.0,0.0,1.147551e-06,0.047759,0.062054,0.003924,0.766234,0.030242,0.092593,...,0.512195,0.679487,0.003444,0.485294,0.868852,0.416667,0.085106,0.101695,0.608108,0.426230
4,0.0,1.0,0.0,1.475423e-06,0.067503,0.061830,0.000228,0.714286,0.111712,0.166667,...,0.475610,0.666667,0.001359,0.470588,0.836066,0.416667,0.106383,0.135593,0.594595,0.377049
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114595,1.0,0.0,1.0,9.963934e-01,0.063064,0.109624,0.001109,0.272727,0.101046,0.092593,...,0.268293,0.525641,0.004996,0.294118,0.573770,0.183333,0.234043,0.372881,0.513514,0.245902
114596,1.0,0.0,1.0,9.986983e-01,0.087362,0.121108,0.018650,0.324675,0.091576,0.129630,...,0.353659,0.666667,0.009412,0.382353,0.721311,0.300000,0.191489,0.237288,0.540541,0.360656
114597,1.0,0.0,1.0,9.993445e-01,0.105254,0.146113,0.122325,0.233766,0.237749,0.092593,...,0.341463,0.602564,0.007087,0.411765,0.721311,0.233333,0.170213,0.254237,0.567568,0.311475
114598,1.0,0.0,1.0,9.993556e-01,0.207636,0.203465,0.036061,0.324675,0.176087,0.259259,...,0.463415,0.794872,0.007127,0.514706,0.803279,0.466667,0.255319,0.152542,0.472973,0.491803


In [ ]:
# tabnet 이라는 변수 선택 딥러닝 기법도 활용해보기

In [ ]:
def calculate_difference(row):
    current_date = row['date']
    previous_date = current_date - 2  # 2달 전의 데이터를 찾기

    # 이전 시점의 데이터를 찾아서 존재하면 차이를 계산, 없으면 0으로 처리
    previous_data = X[(X['date'] == previous_date) & (X['gender'] == row['gender']) & 
                       (X['age'] == row['age']) & (X['region'] == row['region'])]

    if not previous_data.empty:
        return pd.Series(row[['la01r', 'la02r', 'la03r', 'la04r', 'la05r', 'la06r', 'la07r']].values - previous_data[['la01r', 'la02r', 'la03r', 'la04r', 'la05r', 'la06r', 'la07r']].values[0],
                         index=['diff_la01r', 'diff_la02r', 'diff_la03r', 'diff_la04r', 'diff_la05r', 'diff_la06r', 'diff_la07r'])
    else:
        return pd.Series([0, 0, 0, 0, 0, 0, 0], index=['diff_la01r', 'diff_la02r', 'diff_la03r', 'diff_la04r', 'diff_la05r', 'diff_la06r', 'diff_la07r'])

X = pd.concat([X, X.apply(calculate_difference, axis=1)], axis=1)

print(X)

In [ ]:
X['abs_1year_diff_sum'] = X[['diff_la01r', 'diff_la02r', 'diff_la03r', 'diff_la04r', 'diff_la05r', 'diff_la06r', 'diff_la07r']].abs().sum(axis=1)

In [ ]:
cols_to_drop = [col for col in X.columns if col.startswith('la') or col.startswith('diff')]
X = X.drop(cols_to_drop, axis=1)